## Ejercicio 2
Utilizar Simpy para simular una sección del ejercicio 1 a elección

In [37]:
from generators import LXM
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns; sns.set()
import simpy
import scipy

from IPython.display import display
from ipywidgets import FloatLogSlider, FloatSlider, interactive


Parametros iniciales

In [38]:
lambdaArribos = 2
lambdaServicio = 0.5
cantidadArribos = 1000
generador = LXM()

Vamos a necesitar generar valores exponenciales, los obtenemos con la transformada inversa.

In [39]:
def exponencial(lambdaArgumento, generador):
    uniforme = generador.generar()
    return - np.log(1 - uniforme) / lambdaArgumento

Definimos el modelo del satelite y la solicitud.

In [40]:
largoColaEnTiempo = []
tiemposDeServicio = []
tiemposDeServicioAfectadosPorLaCola = []
class Satelite:
    def __init__(self, env, servicioDescarga, lambdaArgumento, generador):
        self.env = env
        self.lambdaArgumento = lambdaArgumento
        self.generador = generador
        self.servicioDescarga = servicioDescarga

    def atencion(self, id):
        tiempoAtencion = exponencial(self.lambdaArgumento, self.generador)
        servicio = self.servicioDescarga.request()
        yield servicio

        #print("Satelite en uso por la solicitud con id " + str(id))
        N = 1
        largoCola = len(self.servicioDescarga.queue)

        if largoCola != 0:
            N = largoCola
        tiempoServicio = tiempoAtencion / N
        #print("El tiempo de atencion obtenido es: " + str(tiempoAtencion)  + ", tiempoServicio: " + str(tiempoServicio))
        #print("En la cola hay " + str(len(self.servicioDescarga.queue)))

        largoColaEnTiempo.append(largoCola)
        tiemposDeServicio.append(tiempoAtencion)
        tiemposDeServicioAfectadosPorLaCola.append(tiempoServicio)

        yield env.timeout(tiempoServicio)
        self.servicioDescarga.release(servicio)

class Solicitud:
    def __init__(self, env, satelite, id):
        self.env = env
        self.satelite = satelite
        self.id = id

    def tratarse(self):
        return self.satelite.atencion(self.id)

Creamos la funcion que se va a encargar de generar las solicitudes. Estas son nuevos procesos que van a intentar usar el servicio del satelite.

A medida que van llegando de avanza el tiempo del ambiente.

In [41]:
tiempoEntreArribos = []
tiemposAcumulados = []

def generarArribos(lambdaArgumento, generador, cantidadArribos, env, satelite):
    tiempoAcumulado = 0
    id = 0
    while (id < cantidadArribos):
        tiempo = exponencial(lambdaArgumento, generador)
        nuevaSolicitud = Solicitud(env, satelite, id)
        id = 1 + id
        env.process(nuevaSolicitud.tratarse())
        yield env.timeout(tiempo)

        tiempoAcumulado += tiempo
        tiemposAcumulados.append(tiempoAcumulado)
        tiempoEntreArribos.append(tiempo)

    #print("Se crearon un total de " + str(id) + " solicitudes")



Establecemos las configuraciones iniciales y ya podemos simular...

In [42]:
env = simpy.Environment()
def explore_config(lambdaArribos, lambdaServicio, cantidadArribos=100_000):
    servicioDescarga = simpy.Resource(env, capacity=1)
    satelite = Satelite(env,servicioDescarga,lambdaServicio,generador)

    env.process(generarArribos(lambdaArribos, generador, cantidadArribos, env, satelite))

    env.run()

    f, axis = plt.subplots(
        nrows=2,
        ncols=3,
        dpi=100,
        figsize=(24, 8),
        gridspec_kw={"width_ratios": [1, 1, 2]},
    )

    axis[0,0].set_title("Densidad del tiempo entre arribos")
    p = sns.kdeplot(tiempoEntreArribos,cut=0,ax=axis[0,0])


    axis[0,1].step(tiemposAcumulados,range(len(tiemposAcumulados)),where= 'post' ,label='λ=' + str(lambdaArribos))
    axis[0,1].legend()
    axis[0,1].set_title("Arribos a lo largo del tiempo - Poisson")


    axis[0,2].set_title("Comportamiento del largo de la cola")
    p = sns.kdeplot(largoColaEnTiempo,cut=0,ax=axis[0,2])

    x,y = p.get_lines()[0].get_data()

    cdf = scipy.integrate.cumtrapz(y, x, initial=0)
    medio = np.abs(cdf-0.5).argmin()

    x_mediana = x[medio]
    y_mediana = y[medio]

    axis[0,2].vlines(x_mediana, 0, y_mediana)
    axis[0,2].set_title("Comportamiento del largo de la cola")
    axis[0,2].set_xlabel("Largo de la cola")
    axis[0,2].set_ylabel("Probabilidad")

    axis[1,0].set_title("Densidad del tiempo de servicio")
    p = sns.kdeplot(tiemposDeServicio,cut=0, ax=axis[1,0])


    axis[1,1].set_title("Densidad del tiempo de servicio afectada por la cola")
    p = sns.kdeplot(tiemposDeServicioAfectadosPorLaCola,cut=0,ax=axis[1,1])

    f.suptitle(f"N={cantidadArribos}  $\lambda$ arribos={lambdaArribos}  $\lambda$ servicio={lambdaServicio}")

    plt.show()
    plt.close()

In [43]:
inter = interactive(
    explore_config,
    {'manual': True},
    lambdaArribos=FloatSlider(value=2, min=0.1, max=10, step=0.1, description="λ Arribos"),
    lambdaServicio=FloatSlider(value=0.5, min=0.01, max=10, step=0.01, description="λ Servicio"),
    cantidadArribos=FloatLogSlider(value=1_000, base=10, min=2, max=8, step=1, description="Iterations", readout_format="d"),
)
display(inter)

interactive(children=(FloatSlider(value=2.0, description='λ Arribos', max=10.0, min=0.1), FloatSlider(value=0.…

In [44]:
# Para reiniciar la historia
tiempoEntreArribos = []
tiemposAcumulados = []
largoColaEnTiempo = []
tiemposDeServicio = []
tiemposDeServicioAfectadosPorLaCola = []